In [15]:
import csv
import urllib
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse
import random

# Open the CSV file in read mode
with open('/Users/tahers/Documents/SE_NPM_packages/random_packages_20k.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)

    # Handle potential header row
    is_header = True
    package_names = []
    for row in reader:
        if is_header:
            is_header = False  # Skip the header row if present
            continue
        package_names.append(row[0])  # Assuming package names are in the first column

packages = package_names[10000:]
print(len(packages))
output_filename = "synk_data_batch2.csv"

def get_package_data(package):
    retry_attempts = 3  # Number of retry attempts
    for _ in range(retry_attempts):
        try:
            url = 'https://snyk.io/advisor/npm-package/{}'.format(urllib.parse.quote_plus(package))
            print("pkg:", package, "-", "GET", url)
            # Set up Selenium WebDriver
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            driver = webdriver.Chrome(options=options)
            driver.get(url)

            # Wait for the page to fully load
            time.sleep(0.38)

            # Extract HTML content after waiting
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, "html.parser")

            container_elements = soup.find_all('div', class_='container')
            values_list = []

            for container in container_elements:
                values = container.find_all('div', class_='item')
                for value in values:
                    span = value.find('span')
                    if span:
                        values_list.append(span.text.strip())
            latest_version = license = health_score = security_recommendation = popularity = maintainence = community = gitHub_stars = forks = contributors = last_release = last_commit = readme_file = age = dependencies = versions = install_size = num_of_files = maintainers = TS_typings = critical_vul_count = high_vul_count = medium_vul_count = low_vul_count = None
            if values_list:
                latest_version = values_list[0].replace('Latest version published', '').strip()
                license = values_list[1].replace('License: ', '').strip()

            number_div = soup.find('div', class_='number')
            if number_div:
                health_score = number_div.find('span').text.strip()[:3].rstrip()

            ul_element = soup.find('ul', class_='scores')
            if ul_element:
                spans = ul_element.find_all('span', class_='vue--pill__body')
                security_recommendation = spans[0].text.strip()
                popularity = spans[1].text.strip()
                maintainence = spans[2].text.strip()
                community = spans[3].text.strip()

            stats_dl = soup.find_all('dl', class_='stats stats--fluid')
            span_values = []
            for dl in stats_dl:
                spans = dl.find_all('span')
                span_values += [span.text.strip() for span in spans]
            if span_values:
                gitHub_stars = span_values[1]
                forks = span_values[3]
                contributors = span_values[5]
                last_release = span_values[-3]
                last_commit = span_values[-1]

            stats_grid = soup.find_all('dl', class_='stats stats--grid')
            stats_grid_values = []
            for grid in stats_grid:
                grids = grid.find_all('span')
                stats_grid_values += [grid1.text.strip() for grid1 in grids]
            if stats_grid_values:
                readme_file = stats_grid_values[1]
                age = stats_grid_values[11]
                dependencies = stats_grid_values[13]
                versions = stats_grid_values[15]
                install_size = stats_grid_values[17]
                num_of_files = stats_grid_values[21]
                maintainers = stats_grid_values[23]
                TS_typings = stats_grid_values[25]

            # Find the table with class vue--security-severity-table
            table = soup.find('table', class_='vue--security-severity-table')
            vulnerabilities_count = []
            if table:
                # Find the row with class vue--security-severity-table__highlighted
                highlighted_row = table.find('tr', class_='vue--security-severity-table__highlighted')
                if highlighted_row:
                    # Find the ul with class vue--severity within the highlighted row
                    ul = highlighted_row.find('ul', class_='vue--severity')
                    if ul:
                        # Find all li tags within the ul
                        li_tags = ul.find_all('li')
                        for li in li_tags:
                            # Find all div tags with class vue--severity__count within each li
                            div_tags = li.find_all('div', class_='vue--severity__count')
                            for div in div_tags:
                                # Find all span tags within each div and extract their text
                                span_tags = div.find_all('span')
                                for span in span_tags:
                                    vulnerabilities_count.append(span.text.strip())
            if vulnerabilities_count:
                critical_vul_count = vulnerabilities_count[0]
                high_vul_count = vulnerabilities_count[1]
                medium_vul_count = vulnerabilities_count[2]
                low_vul_count = vulnerabilities_count[3]

            driver.quit()

            if any(value == 'pending...' or value.strip() == '?' for value in [latest_version, license, health_score]):
                    print("Retrying for package:", package)
                    continue  # Retry for this package
            else:
                    return [package, latest_version, license, health_score, security_recommendation, popularity, maintainence, community, gitHub_stars, forks, contributors, last_release, last_commit, readme_file, age, dependencies, versions, install_size, num_of_files, maintainers, TS_typings, critical_vul_count, high_vul_count, medium_vul_count, low_vul_count]

        except Exception as e:
            print("Error occurred while processing package:", package)
            print(e)
            return None

    print("Failed to retrieve data for package after", retry_attempts, "attempts:", package)
    return None

cnt=0
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Package', 'Latest Version', 'License', 'Health Score', 'Security Recommendation', 'Popularity', 'Maintainence', 'Community', 'GitHub Stars', 'Forks', 'Contributors', 'Last Release', 'Last Commit', 'Readme File', 'Age', 'Dependencies', 'Versions', 'Install Size', '# of Files', 'Maintainers', 'TS Typings', 'critical_vul_count', 'high_vul_count', 'medium_vul_count', 'low_vul_count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for package in packages:
        cnt+=1
        package_data = get_package_data(package)
        if package_data:
            writer.writerow(dict(zip(fieldnames, package_data)))
        print(cnt)
print("Data written to", output_filename)

10000
pkg: strappyjs-ccl - GET https://snyk.io/advisor/npm-package/strappyjs-ccl
1
pkg: avivbh-another-proto - GET https://snyk.io/advisor/npm-package/avivbh-another-proto
2
pkg: awesome-css-spinners - GET https://snyk.io/advisor/npm-package/awesome-css-spinners
3
pkg: ember-cli-validations-errors - GET https://snyk.io/advisor/npm-package/ember-cli-validations-errors
4
pkg: byclient - GET https://snyk.io/advisor/npm-package/byclient
5
pkg: fh-jira - GET https://snyk.io/advisor/npm-package/fh-jira
6
pkg: huangenci08 - GET https://snyk.io/advisor/npm-package/huangenci08
7
pkg: tommy_addon_api - GET https://snyk.io/advisor/npm-package/tommy_addon_api
8
pkg: worker-static-loader - GET https://snyk.io/advisor/npm-package/worker-static-loader
9
pkg: number-formatter-example - GET https://snyk.io/advisor/npm-package/number-formatter-example
10
pkg: eslint-config-nikola - GET https://snyk.io/advisor/npm-package/eslint-config-nikola
Error occurred while processing package: eslint-config-nikola
